<center><h1> Phishing detection </h1></center>

<a id='index-of-content'></a>
<h2> 1. Index </h2>

<a id='lectura-de-datos'></a>
<h2> 2. Data reading </h2>

In [2]:
import pandas

urls = pandas.read_csv('dataset_url.csv', header=0)

In [13]:
from sklearn import preprocessing

atributos = urls.loc[:, 'qty_dot_url':'url_shortened']  # selección de las columnas de atributos
objetivo = urls['phishing']  # selección de la columna objetivo

In [ ]:
#### To analyse message content ####
data = pandas.read_csv('phishing.csv', header=0)
from sklearn import preprocessing

atributos = data.loc[:, 'Total Number of Characters C':'Unique Words']
objetivo = data['Phishing Status']

<a id='codificacion-de-atributos'></a>
<h2> 3. Enconding attributes </h2>

In [14]:
def codifica(atributos, objetivo):
    codificador_atributos = preprocessing.OrdinalEncoder()
    codificador_atributos.fit(atributos)
    atributos_codificados = codificador_atributos.transform(atributos)
    codificador_objetivo = preprocessing.LabelEncoder()
    objetivo_codificado = codificador_objetivo.fit_transform(objetivo)
    return atributos_codificados, objetivo_codificado

In [15]:
atributos_codificados, objetivo_codificado = codifica(atributos,objetivo)

<a id='separacion-de-conjuntos'></a>
<h2> 4. Separation of training and testing set </h2>

In [16]:
print(pandas.Series(objetivo).value_counts(normalize=True))

0    0.65428
1    0.34572
Name: phishing, dtype: float64


In [17]:
from sklearn import model_selection

(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = model_selection.train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos_codificados, objetivo_codificado,
        # Valor de la semilla aleatoria, para que el muestreo sea reproducible,
        # a pesar de ser aleatorio
        random_state=42,
        # Tamaño del conjunto de prueba
        test_size=.33,
        # Estratificamos respecto a la distribución de valores en la variable objetivo
        stratify=objetivo_codificado)

In [18]:
def imprime_proporciones(objetivo_prueba,objetivo_entrenamiento):
    codificador_atributos = preprocessing.OrdinalEncoder()
    codificador_atributos.fit(atributos)
    atributos_codificados = codificador_atributos.transform(atributos)
    codificador_objetivo = preprocessing.LabelEncoder()
    objetivo_codificado = codificador_objetivo.fit_transform(objetivo)
    print("Proporción de clases en el vector de objetivos  de prueba:")
    print(pandas.Series(
        codificador_objetivo.inverse_transform(objetivo_prueba)
      ).value_counts(normalize=True))
    print("Proporción de clases en el vector de objetivos de entrenamiento:")
    print(pandas.Series(
        codificador_objetivo.inverse_transform(objetivo_entrenamiento)
      ).value_counts(normalize=True))

In [19]:
imprime_proporciones(objetivo_prueba,objetivo_entrenamiento)

Proporción de clases en el vector de objetivos  de prueba:
0    0.65427
1    0.34573
dtype: float64
Proporción de clases en el vector de objetivos de entrenamiento:
0    0.654286
1    0.345714
dtype: float64


<a id='modelos-de-clasificacion'></a>
<h2> 4. Classification models </h2>

<a id='knn-5-vecinos'></a>
<h2> 4.1. KNN Classifier </h2>

In [22]:
from sklearn import neighbors
def score_kNN(atributos_entrenamiento, objetivo_entrenamiento,atributos_prueba, objetivo_prueba, k):    
    clasif_kNN = neighbors.KNeighborsClassifier(n_neighbors=k, metric='hamming')
    clasif_kNN.fit(atributos_entrenamiento, objetivo_entrenamiento)

    return clasif_kNN.score(atributos_prueba, objetivo_prueba)

In [23]:
score_kNN(atributos_entrenamiento, objetivo_entrenamiento,atributos_prueba, objetivo_prueba, 5)

0.948690777329596

In [42]:
clasif_kNN = neighbors.KNeighborsClassifier(n_neighbors=5, metric='hamming')
clasif_kNN.fit(atributos_entrenamiento, objetivo_entrenamiento)
ejemplo_nuevo = atributos_prueba[0]
prediction = clasif_kNN.predict([ejemplo_nuevo])
print(prediction)

[1]


<a id='decision-tree'></a>
<h2> 4.2. Decission Tree Classifier</h2>

In [44]:
from sklearn.tree import DecisionTreeClassifier
def score_tree_classifier(atributos_entrenamiento, objetivo_entrenamiento,atributos_prueba, objetivo_prueba):
    clasif_Tree = DecisionTreeClassifier(random_state=0)
    clasif_Tree.fit(atributos_entrenamiento, objetivo_entrenamiento)

    return clasif_Tree.score(atributos_prueba, objetivo_prueba)

In [45]:
score_tree_classifier(atributos_entrenamiento, objetivo_entrenamiento,atributos_prueba, objetivo_prueba)

0.9506050454638681

<a id='neuronal-network'></a>
<h2> 4.3. Neural Network Classifier</h2>

In [46]:
from sklearn.neural_network import MLPClassifier
def score_neuronal_network(atributos_entrenamiento, objetivo_entrenamiento,atributos_prueba, objetivo_prueba):
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(2, 5), random_state=1)
    clf.fit(atributos_entrenamiento, objetivo_entrenamiento)
    
    return clf.score(atributos_prueba, objetivo_prueba)

In [47]:
score_neuronal_network(atributos_entrenamiento, objetivo_entrenamiento,atributos_prueba, objetivo_prueba)

/Users/maribelrb/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.634921720106652

<a id='random-forest'></a>
<h2>4.4. Random Forest Classifier</h2>

In [50]:
from sklearn.ensemble import RandomForestClassifier
def score_random_forest(atributos_entrenamiento, objetivo_entrenamiento,atributos_prueba, objetivo_prueba):
    clasif_RF = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
    clasif_RF.fit(atributos_entrenamiento, objetivo_entrenamiento)
    return clasif_RF.score(atributos_prueba, objetivo_prueba)

In [51]:
score_random_forest(atributos_entrenamiento, objetivo_entrenamiento,atributos_prueba, objetivo_prueba)

0.8642578792643741

<a id='logistic-regresion'></a>
<h2>4.5. Logistic Regression Classifier</h2>

In [52]:
from sklearn.linear_model import LogisticRegression
def score_Logistic_Regression(atributos_entrenamiento, objetivo_entrenamiento,atributos_prueba, objetivo_prueba):
    clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial')
    clf.fit(atributos_entrenamiento, objetivo_entrenamiento)
    return clf.score(atributos_prueba, objetivo_prueba)

In [53]:
score_Logistic_Regression(atributos_entrenamiento, objetivo_entrenamiento,atributos_prueba, objetivo_prueba)

/Users/maribelrb/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8836398441238805